In [15]:
import pandas as pd
mapper = {
    'pri_neigh':'neighborhood',
    'nearestTrainStopDist_sb':'nearestTransitStopDist_sb',
    'nearestTrainStopDist_nsb':'nearestTransitStopDist_nsb',
    'Neighborhoods - Analysis Boundaries':'neighborhood',
    'nearestBusStopDist_sb':'nearestTransitStopDist_sb',
    'nearestBusStopDist_nsb':'nearestTransitStopDist_nsb'
}

In [23]:
sf = pd.read_csv('sf_ratios_081117.csv')
sf.rename(columns=mapper,inplace=True)
sf.head()

,neighborhood,index_sb,nearestCoffeeDist_sb,nearestTransitStopDist_sb,index_nsb,nearestCoffeeDist_nsb,nearestTransitStopDist_nsb,nsbRatio
0,Castro/Upper Market,1,0.000207,0.000064,11,0.001297,0.000552,11.000000
1,Chinatown,2,0.001185,0.000588,6,0.001225,0.000312,3.000000
2,Financial District/South Beach,27,0.000829,0.000461,26,0.000739,0.000661,0.962963
3,Inner Sunset,2,0.001513,0.000792,5,0.000605,0.000821,2.500000
4,Lakeshore,1,0.000000,0.000822,1,0.000000,0.001857,1.000000


In [22]:
chi = pd.read_csv('chicago_ratios_081117.csv')
chi.rename(columns=mapper,inplace=True)
chi.head()

,neighborhood,index_sb,nearestTransitStopDist_sb,nearestCoffeeDist_sb,index_nsb,nearestTransitStopDist_nsb,nearestCoffeeDist_nsb,nsbRatio
0,Albany Park,2,0.008307,0.006013,2.0,0.006201,0.006013,1.0
1,Andersonville,1,0.009828,0.004972,1.0,0.009682,0.001469,1.0
2,Belmont Cragin,2,0.043445,0.002069,2.0,0.028794,0.020678,1.0
3,Beverly,1,0.000000,0.023791,1.0,0.054787,0.023791,1.0
4,Boystown,1,0.009095,0.001523,2.0,0.007175,0.002895,2.0


In [24]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
chi_noNeigh = chi[[ x for x in chi.columns if x if 'neighborhood' != x ]]
sf_noNeigh = sf[[ x for x in sf.columns if x if 'neighborhood' != x ]]

In [41]:
sim = pd.DataFrame(cosine_similarity(chi_noNeigh,sf_noNeigh))
sim.columns = sf['neighborhood']
sim.index = chi['neighborhood']

In [48]:
sim

neighborhood,Castro/Upper Market,Chinatown,Financial District/South Beach,Inner Sunset,Lakeshore,Lone Mountain/USF,Marina,Mission Bay,Nob Hill,Noe Valley,...,Outer Richmond,Pacific Heights,Potrero Hill,Presidio Heights,Russian Hill,South of Market,Sunset/Parkside,Tenderloin,West of Twin Peaks,Western Addition
neighborhood,,,,,,,,,,,,,,,,,,,,,
Albany Park,0.748410,0.904754,0.950882,0.926359,0.962243,0.793486,0.994127,0.987699,0.888882,0.770523,...,0.748410,0.999992,0.757145,0.793486,0.812355,0.832343,0.748410,0.879236,0.904754,0.962246
Andersonville,0.851822,0.907234,0.830880,0.923781,0.999965,0.889269,0.927144,0.929730,0.962219,0.870359,...,0.851822,0.962219,0.859179,0.889268,0.904503,0.789758,0.851822,0.828889,0.907234,0.999976
Belmont Cragin,0.748287,0.904608,0.950725,0.926211,0.962089,0.793362,0.993974,0.987542,0.888741,0.770400,...,0.748288,0.999832,0.757021,0.793361,0.812223,0.832206,0.748289,0.879092,0.904608,0.962112
Beverly,0.851269,0.906648,0.830339,0.923183,0.999332,0.888696,0.926539,0.929135,0.961593,0.869800,...,0.851269,0.961593,0.858621,0.888691,0.903919,0.789245,0.851274,0.828350,0.906646,0.999335
Boystown,0.962243,0.952374,0.719418,0.954432,0.962243,0.980190,0.841187,0.880342,0.999993,0.971530,...,0.962243,0.888883,0.966014,0.980190,0.986433,0.854057,0.962243,0.868108,0.952374,0.962248
Bucktown,0.691697,0.884886,0.978722,0.908050,0.927168,0.738864,0.999997,0.990120,0.841188,0.714769,...,0.691697,0.994132,0.700797,0.738864,0.758784,0.831893,0.691697,0.880893,0.884886,0.927172
Chatham,0.851847,0.907260,0.830905,0.923807,0.999994,0.889293,0.927169,0.929756,0.962246,0.870384,...,0.851847,0.962246,0.859205,0.889293,0.904529,0.789782,0.851847,0.828913,0.907260,0.999998
East Village,0.962248,0.952379,0.719422,0.954437,0.962248,0.980194,0.841190,0.880346,0.999998,0.971535,...,0.962248,0.888887,0.966019,0.980194,0.986438,0.854062,0.962248,0.868112,0.952379,0.962249
Edgewater,0.993869,0.944991,0.626062,0.938237,0.904533,0.999404,0.758786,0.818572,0.986438,0.997239,...,0.993869,0.812361,0.995339,0.999404,0.999998,0.861800,0.993869,0.860216,0.944991,0.904532
